In [3]:
from logger import setup_logger
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
import hashlib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import random
import sqlite3
from utils.helper_functions import *
from datetime import datetime, timedelta

def get_data(start_date, end_date):
    # Configure ChromeOptions for headless browsing
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")  # This line can be important in certain environments
    options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
    # Initialize the Chrome WebDriver with the specified options
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.bovada.lv/sports/football/nfl")
    # wait for the page to load
    time.sleep(10)
    driver.implicitly_wait(10)
    # get the HTML source
    html = driver.page_source
    # create a BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")
    # close the driver
    driver.quit()

    data = []
    sections = soup.find_all("section", {"class":"coupon-content more-info"})#soup.find_all("section", {"class":"coupon-content more-info"})
    for game in sections:
        try:
            item = str(game).split('>')
            info = [x.split('<')[0].strip() for x in item if not x.startswith("<")]
            data.append(info)
        except Exception as e:
            pass

    df = pd.DataFrame(data)

    df["Home Spread"] = df.apply(lambda row: concat_values(row[10], row[11]), axis=1)
    df["Away Spread"] = df.apply(lambda row: concat_values(row[12], row[13]), axis=1)
    df["total_home"] = df.apply(lambda row: concat_values(row[16], row[17], row[18]), axis=1)
    df["total_away"] = df.apply(lambda row: concat_values(row[19], row[20], row[21]), axis=1)
    #drop columns
    df.drop(columns = [3, 4, 5, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22], inplace=True)
    columns = ["date", "time", "bets", "home_team", "away_team", "home_win", "away_win", "home_spread", "away_spread", "total_over", "total_under"]
    df.columns = columns

    #remove plus from bets
    df['bets'] = df['bets'].apply(lambda x: x[2:])

    #date operations
    #filter data for date
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')  # Adjust the format if needed
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Adjust the format if needed
        # Ensure the 'date' column in df is of type datetime.date
    
    # Ensure the 'date' column in df is of type datetime
    df['date'] = pd.to_datetime(df['date'])

    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    #create day of the week column
    df["day"] = df['date'].dt.strftime('%A')
    #set back to string
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df.reset_index(inplace=True, drop=True)

    # Applying the conversion to the 'win_home' and "Away Win" columns
    df['home_win'] = df['home_win'].apply(convert_to_int)
    df["away_win"] = df["away_win"].apply(convert_to_int)
    #ranking
    home = df[["home_team", 'home_win']].rename(columns={'home_team': 'team', 'home_win': 'odds'})
    away = df[['away_team', "away_win"]].rename(columns={'away_team': 'team', "away_win": 'odds'})
    combined = pd.concat([home, away]).sort_values('odds', ascending=False)
    combined['index'] = combined.index
    combined.index = range(0, 2*len(combined), 2)
    df['points'] = None
    # Iterating over the combined DataFrame to assign ranks
    for i, x in combined.iterrows():
        df.at[x['index'], 'points'] = (i-len(combined))/2+1
    current_df = df.sort_values('points', ascending=False)
    #add game id
    current_df["game_id"] = current_df.apply(generate_game_id, axis=1)
    #change column order
    current_df = current_df[['date', 'day', 'time', 'bets', 'home_team', 'away_team', 'points', 'home_win', 'away_win', 'home_spread', 'away_spread', 'total_over', 'total_under', 'game_id']]
    log_data = current_df[['game_id', 'date', 'home_team', 'away_team', 'home_win', 'away_win', 'points']]
    log_data_if_changed(log_data)

    return current_df

def generate_matchups(df):
    # Ensure DateTime is properly formatted
    df['DateTime'] = pd.to_datetime(df['date'])

    # Sort the DataFrame by DateTime to get matchups from soonest to latest
    sorted_df = df.sort_values(by='DateTime')

    # Prepare data for the DataTable
    matchups_data = []

    team_conversion_dict = {
    "Houston Texans": "HOU",
    "New York Jets": "NYJ",
    "Denver Broncos": "DEN",
    "Baltimore Ravens": "BAL",
    "Jacksonville Jaguars": "JAX",
    "Philadelphia Eagles": "PHI",
    "New Orleans Saints": "NO",
    "Carolina Panthers": "CAR",
    "Las Vegas Raiders": "LV",
    "Cincinnati Bengals": "CIN",
    "Miami Dolphins": "MIA",
    "Buffalo Bills": "BUF",
    "Indianapolis Colts": "IND",
    "Minnesota Vikings": "MIN",
    "Washington Commanders": "WSH",
    "New York Giants": "NYG",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "New England Patriots": "NE",
    "Tennessee Titans": "TEN",
    "Dallas Cowboys": "DAL",
    "Atlanta Falcons": "ATL",
    "Chicago Bears": "CHI",
    "Arizona Cardinals": "ARI",
    "Los Angeles Chargers": "LAC",
    "Cleveland Browns": "CLE",
    "Los Angeles Rams": "LAR",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TB",
    "Kansas City Chiefs": "KC",
    'San Francisco 49ers': "SF",
    'Pittsburgh Steelers': "PIT"
    }

    for _, row in sorted_df.iterrows():
        home_team = row['home_team']
        away_team = row['away_team']
        points = row['points']

        # Determine the favored team
        projected_winner = home_team if row['home_win'] < row['away_win'] else away_team

        # Add row data
        matchups_data.append({
            "game_id": f"{team_conversion_dict[home_team]}{team_conversion_dict[away_team]}",
            "matchup": f"{home_team} vs {away_team}",
            "time": row['DateTime'].strftime('%H:%M %p'),
            "projected_winner": projected_winner,
            "ranking": points
        })

    return matchups_data

def get_espn_expert_data():
    # Function to transform the game string
    def transform_game(game):
        try:
            teams = game.split(' at ')
            return teams[0] + teams[1]
        except:
            teams = game.split(' VS ')
            return teams[0] + teams[1]
    try:
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")  # This line can be important in certain environments
        options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
        # Initialize the Chrome WebDriver with the specified options
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.espn.com/nfl/picks")
        #time.sleep(10)
        driver.implicitly_wait(10)
        # get the HTML source
        html = driver.page_source
        # create a BeautifulSoup object
        soup = BeautifulSoup(html, "html.parser")
        # close the driver
        driver.quit()

        week = soup.find('h1', class_='headline headline__h1 dib').get_text(strip=True).split('- ')[1]

        # Extract game details
        games = []
        game_rows = soup.select('.Table--fixed-left .Table__TBODY .Table__TR')
        for row in game_rows:
            game_info_element = row.select_one('.wrap-competition a')
            game_time_element = row.select_one('.competition-dates')
            if game_info_element and game_time_element:
                game_info = game_info_element.text
                game_time = game_time_element.text
                games.append((game_info, game_time))

        # Extract expert names
        experts = []
        expert_headers = soup.select('.Table__Scroller .Table__THEAD .Table__TH')
        for header in expert_headers:
            expert_name_element = header.select_one('div')
            if expert_name_element:
                expert_name = expert_name_element.text.strip()
                experts.append(expert_name)

        # Extract picks
        picks = []
        pick_rows = soup.select('.Table__Scroller .Table__TBODY .Table__TR')
        for row in pick_rows:
            pick_row = []
            pick_cells = row.select('.Table__TD')
            for cell in pick_cells:
                team_logo = cell.select_one('img')
                if team_logo:
                    # Extract the team abbreviation from the image URL
                    team = team_logo['src'].split('/')[-1].split('.')[0]
                else:
                    team = None
                pick_row.append(team)
            picks.append(pick_row)

        # Create DataFrame
        data = {'Game': [game[0] for game in games], 'Time': [game[1] for game in games]}
        for i, expert in enumerate(experts):
            data[expert] = [pick[i] for pick in picks]

        data['Game'].append(None)
        data['Time'].append(None)

        df = pd.DataFrame(data)
        df.dropna(subset=["Game"], inplace=True)

        df['week'] = week

        convert_dict = {
            "min": "Vikings", "phi": "Eagles", "bal": "Ravens", "det": "Lions", "mia": "Dolphins",
            "nyj": "Jets", "atl": "Falcons", "gb": "Packers", "hou" : "Texans", "lac": "Chargers",
            "buf": "Bills", "den": "Broncos", "kc": "Chiefs", "chi": "Bears", "sf": "49ers", "pit": "Steelers",
            "no": "Saints", "cin": "Bengals", "ne": "Patriots", "wsh": "Commanders", "ari": "Cardinals", 
            "lar": "Rams"
        }

        for ix, row in df.iterrows():
            values = row.to_list()[2:]
            values = [value for value in values if value is not None]
            values_len = len(values)
            values_dict = {}
            for value in values:
                if value not in values_dict.keys():
                    values_dict[value] = 1
                else:
                    values_dict[value] += 1
            #sorting
            values_dict = dict(sorted(values_dict.items(), key=lambda item: item[1], reverse=True))
            top_key = next(iter(values_dict))
            if top_key in convert_dict:
                converted_key = convert_dict[top_key]
            else:
                converted_key = top_key
            pct = int(values_dict[top_key]/values_len*100)
            message = f"{pct}% of experts chose {converted_key}"
            df.loc[ix, "pct"] = pct
            df.loc[ix, "message"] = message

        df["game_id"] = df["Game"].apply(transform_game)
        return df[["game_id", "week", "Game", "Time", "pct", "message"]]
    except Exception as e:
        logger.exception(f"get espn data, {e}")

In [ ]:
def get_espn_expert_data():
    # Function to transform the game string
    def transform_game(game):
        try:
            teams = game.split(' at ')
            return teams[0] + teams[1]
        except:
            teams = game.split(' VS ')
            return teams[0] + teams[1]

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")  # This line can be important in certain environments
    options.set_capability('goog:loggingPrefs', {'browser': 'SEVERE'})
    # Initialize the Chrome WebDriver with the specified options
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.espn.com/nfl/picks")
    #time.sleep(10)
    driver.implicitly_wait(10)
    # get the HTML source
    html = driver.page_source
    # create a BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")
    # close the driver
    driver.quit()

    week = soup.find('h1', class_='headline headline__h1 dib').get_text(strip=True).split('- ')[1]

    # Extract game details
    games = []
    game_rows = soup.select('.Table--fixed-left .Table__TBODY .Table__TR')
    for row in game_rows:
        game_info_element = row.select_one('.wrap-competition a')
        game_time_element = row.select_one('.competition-dates')
        if game_info_element and game_time_element:
            game_info = game_info_element.text
            game_time = game_time_element.text
            games.append((game_info, game_time))

    # Extract expert names
    experts = []
    expert_headers = soup.select('.Table__Scroller .Table__THEAD .Table__TH')
    for header in expert_headers:
        expert_name_element = header.select_one('div')
        if expert_name_element:
            expert_name = expert_name_element.text.strip()
            experts.append(expert_name)

    # Extract picks
    picks = []
    pick_rows = soup.select('.Table__Scroller .Table__TBODY .Table__TR')
    for row in pick_rows:
        pick_row = []
        pick_cells = row.select('.Table__TD')
        for cell in pick_cells:
            team_logo = cell.select_one('img')
            if team_logo:
                # Extract the team abbreviation from the image URL
                team = team_logo['src'].split('/')[-1].split('.')[0]
            else:
                team = None
            pick_row.append(team)
        picks.append(pick_row)

    # Create DataFrame
    data = {'Game': [game[0] for game in games], 'Time': [game[1] for game in games]}
    for i, expert in enumerate(experts):
        data[expert] = [pick[i] for pick in picks]

    data['Game'].append(None)
    data['Time'].append(None)

    df = pd.DataFrame(data)
    df.dropna(subset=["Game"], inplace=True)

    df['week'] = week

    convert_dict = {
        "min": "Vikings", "phi": "Eagles", "bal": "Ravens", "det": "Lions", "mia": "Dolphins",
        "nyj": "Jets", "atl": "Falcons", "gb": "Packers", "hou" : "Texans", "lac": "Chargers",
        "buf": "Bills", "den": "Broncos", "kc": "Chiefs", "chi": "Bears", "sf": "49ers", "pit": "Steelers",
        "no": "Saints", "cin": "Bengals", "ne": "Patriots", "wsh": "Commanders", "ari": "Cardinals", 
        "lar": "Rams"
    }

    ##splitting here
    for ix, row in df.iterrows():
        for col_name, col_value in row.items():
            if col_value in convert_dict.keys():
                df.at[ix, col_name] = convert_dict[col_value]

    return df


def transform_expert_data(df):
    for ix, row in df.iterrows():
        values = row.to_list()[2:]
        values = [value for value in values if value is not None]
        values_len = len(values)
        values_dict = {}
        for value in values:
            if value not in values_dict.keys():
                values_dict[value] = 1
            else:
                values_dict[value] += 1
        #sorting
        values_dict = dict(sorted(values_dict.items(), key=lambda item: item[1], reverse=True))
        top_key = next(iter(values_dict))
        if top_key in convert_dict:
            converted_key = convert_dict[top_key]
        else:
            converted_key = top_key
        pct = int(values_dict[top_key]/values_len*100)
        message = f"{pct}% of experts chose {converted_key}"
        df.loc[ix, "pct"] = pct
        df.loc[ix, "message"] = message

    df["game_id"] = df["Game"].apply(transform_game)

    return df

In [ ]:
df = get

dict_keys(['min', 'phi', 'bal', 'det', 'mia', 'nyj', 'atl', 'gb', 'hou', 'lac', 'buf', 'den', 'kc', 'chi', 'sf', 'pit', 'no', 'cin', 'ne', 'wsh', 'ari', 'lar'])

In [16]:
row

Game          HOU at DAL
Time          Mon 8:15PM
Bell                None
Bowen                hou
Clay                 hou
Fowler              None
Graziano             dal
Kahler               hou
Martin               hou
Moody                hou
Reid                 hou
Thiry                hou
Wickersham           hou
week             Week 11
Name: 13, dtype: object

In [20]:
for ix, row in df.iterrows():
    for col_name, col_value in row.items():
        if col_value in convert_dict.keys():
            df.at[ix, col_name] = convert_dict[col_value]

In [21]:
df

,Game,Time,Bell,Bowen,Clay,Fowler,Graziano,Kahler,Martin,Moody,Reid,Thiry,Wickersham,week
0,WSH at PHI,Thu 8:15PM,None,Eagles,Eagles,None,Commanders,Commanders,Eagles,Eagles,Eagles,Commanders,Eagles,Week 11
1,GB at CHI,Sun 1:00PM,None,Packers,Packers,None,Packers,Packers,Packers,Packers,Packers,Packers,Packers,Week 11
2,JAX at DET,Sun 1:00PM,None,Lions,Lions,None,Lions,Lions,Lions,Lions,Lions,Lions,Lions,Week 11
3,MIN at TEN,Sun 1:00PM,None,Vikings,Vikings,None,Vikings,Vikings,Vikings,Vikings,Vikings,Vikings,ten,Week 11
4,LV at MIA,Sun 1:00PM,None,Dolphins,Dolphins,None,Dolphins,Dolphins,Dolphins,Dolphins,Dolphins,Dolphins,Dolphins,Week 11
5,LAR at NE,Sun 1:00PM,None,Rams,Rams,None,Rams,Rams,Rams,Rams,Rams,Rams,Rams,Week 11
6,CLE at NO,Sun 1:00PM,None,cle,Saints,None,cle,Saints,Saints,cle,cle,Saints,Saints,Week 11
7,IND at NYJ,Sun 1:00PM,None,Jets,Jets,None,ind,ind,Jets,Jets,ind,ind,Jets,Week 11
8,BAL at PIT,Sun 1:00PM,None,Ravens,Ravens,None,Ravens,Ravens,Ravens,Ravens,Ravens,Ravens,Steelers,Week 11
9,ATL at DEN,Sun 4:05PM,None,Broncos,Falcons,None,Broncos,Falcons,Broncos,Broncos,Falcons,Falcons,Broncos,Week 11


In [19]:
convert_dict[col_value]

'Eagles'

In [5]:
df[["game_id", "week", "Game", "Time", "pct", "message"]]

,game_id,week,Game,Time,pct,message
0,WSHPHI,Week 11,WSH at PHI,Thu 8:15PM,60.0,60% of experts chose Eagles
1,GBCHI,Week 11,GB at CHI,Sun 1:00PM,90.0,90% of experts chose Packers
2,JAXDET,Week 11,JAX at DET,Sun 1:00PM,90.0,90% of experts chose Lions
3,MINTEN,Week 11,MIN at TEN,Sun 1:00PM,80.0,80% of experts chose Vikings
4,LVMIA,Week 11,LV at MIA,Sun 1:00PM,90.0,90% of experts chose Dolphins
5,LARNE,Week 11,LAR at NE,Sun 1:00PM,90.0,90% of experts chose Rams
6,CLENO,Week 11,CLE at NO,Sun 1:00PM,50.0,50% of experts chose Saints
7,INDNYJ,Week 11,IND at NYJ,Sun 1:00PM,50.0,50% of experts chose Jets
8,BALPIT,Week 11,BAL at PIT,Sun 1:00PM,80.0,80% of experts chose Ravens
9,ATLDEN,Week 11,ATL at DEN,Sun 4:05PM,50.0,50% of experts chose Broncos


In [2]:
today = datetime.now()
weekday = today.weekday()  # Monday is 0 and Sunday is 6

# Calculate the start date (Tuesday)
if weekday >= 1:  # If today is Tuesday or after
    start_date = today - timedelta(days=(weekday - 1))
else:  # If today is before Tuesday
    start_date = today - timedelta(days=(weekday + 6))

# Calculate the end date (Monday)
if weekday <= 0:  # If today is Monday
    end_date = today
else:  # If today is after Monday
    end_date = today + timedelta(days=(7 - weekday))

bovada_df = get_data(start_date, end_date)

In [3]:
bovada_df

,date,day,time,bets,home_team,away_team,points,home_win,away_win,home_spread,away_spread,total_over,total_under,game_id
5,2024-11-17,Sunday,1:00 PM,236,Jacksonville Jaguars,Detroit Lions,14.0,525,-800,+14.0 (-115),-14.0 (-105),O 47.0 (-110),U 47.0 (-110),d885e3b557d6a5fefb67827637e37c64
13,2024-11-18,Monday,8:15 PM,211,Houston Texans,Dallas Cowboys,13.0,-360,280,-7.5 (-105),+7.5 (-115),O 42.0 (-110),U 42.0 (-110),8c2655bf8371d234210fe837523a1b05
6,2024-11-17,Sunday,1:00 PM,265,Las Vegas Raiders,Miami Dolphins,12.0,270,-340,+7.0 (-105),-7.0 (-115),O 43.5 (-115),U 43.5 (-105),660f637c3203252bdcbdfa3d922e87fc
10,2024-11-17,Sunday,4:05 PM,216,Seattle Seahawks,San Francisco 49ers,11.0,235,-285,+6.5 (-110),-6.5 (-110),O 47.5 (-110),U 47.5 (-110),5e678bc6b764d5ef990ed6e01db07367
8,2024-11-17,Sunday,1:00 PM,234,Minnesota Vikings,Tennessee Titans,10.0,-270,220,-6.0 (-110),+6.0 (-110),O 39.5 (-110),U 39.5 (-110),ca99ae6e7ff01af95ce56d3f5181f83f
3,2024-11-17,Sunday,1:00 PM,242,Green Bay Packers,Chicago Bears,9.0,-245,205,-5.5 (-110),+5.5 (-110),O 40.0 (-115),U 40.0 (-105),011d50d6b0c91e1eaa0cbefb151633fa
7,2024-11-17,Sunday,1:00 PM,259,Los Angeles Rams,New England Patriots,8.0,-230,190,-4.5 (-115),+4.5 (-105),O 43.5 (-110),U 43.5 (-110),b24a68ca21b5ba6eda380cd67a6546ed
0,2024-11-14,Thursday,8:15 PM,309,Washington Commanders,Philadelphia Eagles,7.0,180,-220,+4.0 (-110),-4.0 (-110),O 49.5 (-110),U 49.5 (-110),e998b8d696a289c8761267714cda8813
4,2024-11-17,Sunday,1:00 PM,228,Indianapolis Colts,New York Jets,6.0,170,-200,+4.0 (-120),-4.0 (EVEN),O 43.5 (-110),U 43.5 (-110),ec3ba976f9a0941d7d7f1b0323961d84
1,2024-11-17,Sunday,1:00 PM,222,Baltimore Ravens,Pittsburgh Steelers,5.0,-175,150,-3.0 (-120),+3.0 (EVEN),O 48.5 (-115),U 48.5 (-105),043237d7a33b0ef08521d93b7211aae6


In [4]:
matchup_df = pd.DataFrame(generate_matchups(bovada_df)).sort_values("ranking", ascending=False)

In [5]:
matchup_df

,game_id,matchup,time,projected_winner,ranking
1,JAXDET,Jacksonville Jaguars vs Detroit Lions,00:00 AM,Detroit Lions,14.0
13,HOUDAL,Houston Texans vs Dallas Cowboys,00:00 AM,Houston Texans,13.0
2,LVMIA,Las Vegas Raiders vs Miami Dolphins,00:00 AM,Miami Dolphins,12.0
3,SEASF,Seattle Seahawks vs San Francisco 49ers,00:00 AM,San Francisco 49ers,11.0
4,MINTEN,Minnesota Vikings vs Tennessee Titans,00:00 AM,Minnesota Vikings,10.0
5,GBCHI,Green Bay Packers vs Chicago Bears,00:00 AM,Green Bay Packers,9.0
6,LARNE,Los Angeles Rams vs New England Patriots,00:00 AM,Los Angeles Rams,8.0
0,WSHPHI,Washington Commanders vs Philadelphia Eagles,00:00 AM,Philadelphia Eagles,7.0
7,INDNYJ,Indianapolis Colts vs New York Jets,00:00 AM,New York Jets,6.0
8,BALPIT,Baltimore Ravens vs Pittsburgh Steelers,00:00 AM,Baltimore Ravens,5.0


In [6]:
expert_df = get_espn_expert_data()

In [7]:
expert_df

,game_id,week,Game,Time,pct,message
0,WSHPHI,Week 11,WSH at PHI,Thu 8:15PM,60.0,60% of experts chose Eagles
1,GBCHI,Week 11,GB at CHI,Sun 1:00PM,90.0,90% of experts chose Packers
2,JAXDET,Week 11,JAX at DET,Sun 1:00PM,90.0,90% of experts chose Lions
3,MINTEN,Week 11,MIN at TEN,Sun 1:00PM,80.0,80% of experts chose Vikings
4,LVMIA,Week 11,LV at MIA,Sun 1:00PM,90.0,90% of experts chose Dolphins
5,LARNE,Week 11,LAR at NE,Sun 1:00PM,90.0,90% of experts chose Rams
6,CLENO,Week 11,CLE at NO,Sun 1:00PM,50.0,50% of experts chose Saints
7,INDNYJ,Week 11,IND at NYJ,Sun 1:00PM,50.0,50% of experts chose Jets
8,BALPIT,Week 11,BAL at PIT,Sun 1:00PM,80.0,80% of experts chose Ravens
9,ATLDEN,Week 11,ATL at DEN,Sun 4:05PM,50.0,50% of experts chose Broncos


In [8]:
expert_df.sort_values("pct", ascending=False)


,game_id,week,Game,Time,pct,message
1,GBCHI,Week 11,GB at CHI,Sun 1:00PM,90.0,90% of experts chose Packers
2,JAXDET,Week 11,JAX at DET,Sun 1:00PM,90.0,90% of experts chose Lions
4,LVMIA,Week 11,LV at MIA,Sun 1:00PM,90.0,90% of experts chose Dolphins
5,LARNE,Week 11,LAR at NE,Sun 1:00PM,90.0,90% of experts chose Rams
3,MINTEN,Week 11,MIN at TEN,Sun 1:00PM,80.0,80% of experts chose Vikings
8,BALPIT,Week 11,BAL at PIT,Sun 1:00PM,80.0,80% of experts chose Ravens
10,SEASF,Week 11,SEA at SF,Sun 4:05PM,80.0,80% of experts chose 49ers
13,HOUDAL,Week 11,HOU at DAL,Mon 8:15PM,80.0,80% of experts chose Texans
0,WSHPHI,Week 11,WSH at PHI,Thu 8:15PM,60.0,60% of experts chose Eagles
6,CLENO,Week 11,CLE at NO,Sun 1:00PM,50.0,50% of experts chose Saints


In [9]:
merged_df = pd.merge(matchup_df, expert_df, on="game_id")
merged_df.drop(columns=["game_id", "time", "pct", "matchup"], inplace=True)
merged_df["IngestTime"] = datetime.now().strftime("%m/%d %H:%M")
merged_df = merged_df[["IngestTime", "week", "Game", "Time", "projected_winner", "ranking", "message"]]


In [10]:
merged_df["ranking"] = merged_df["ranking"]+1

In [11]:
merged_df

,IngestTime,week,Game,Time,projected_winner,ranking,message
0,11/14 16:33,Week 11,JAX at DET,Sun 1:00PM,Detroit Lions,15.0,90% of experts chose Lions
1,11/14 16:33,Week 11,HOU at DAL,Mon 8:15PM,Houston Texans,14.0,80% of experts chose Texans
2,11/14 16:33,Week 11,LV at MIA,Sun 1:00PM,Miami Dolphins,13.0,90% of experts chose Dolphins
3,11/14 16:33,Week 11,SEA at SF,Sun 4:05PM,San Francisco 49ers,12.0,80% of experts chose 49ers
4,11/14 16:33,Week 11,MIN at TEN,Sun 1:00PM,Minnesota Vikings,11.0,80% of experts chose Vikings
5,11/14 16:33,Week 11,GB at CHI,Sun 1:00PM,Green Bay Packers,10.0,90% of experts chose Packers
6,11/14 16:33,Week 11,LAR at NE,Sun 1:00PM,Los Angeles Rams,9.0,90% of experts chose Rams
7,11/14 16:33,Week 11,WSH at PHI,Thu 8:15PM,Philadelphia Eagles,8.0,60% of experts chose Eagles
8,11/14 16:33,Week 11,IND at NYJ,Sun 1:00PM,New York Jets,7.0,50% of experts chose Jets
9,11/14 16:33,Week 11,BAL at PIT,Sun 1:00PM,Baltimore Ravens,6.0,80% of experts chose Ravens
